<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/ML_Spark_RegressaoLinearMultipla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 53.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5b5ff164419ab644562c74bca3e9a5177a99ffdeefb026b46b3c5f23a501e091
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [8]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('Regressao_spark').getOrCreate()

In [11]:
carros = spark.read.csv('Carros.csv', header=True, sep=';', inferSchema=True)
carros.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [12]:
print(carros.count())

32


In [13]:
from pyspark.ml.feature import RFormula

In [17]:
Rformula = RFormula(formula='HP ~ Consumo + Cilindros + Cilindradas', featuresCol='independente', labelCol='dependente')
carrosrf = Rformula.fit(carros).transform(carros)
carrosrf.select('independente','dependente').show(truncate=False)

+------------------+----------+
|independente      |dependente|
+------------------+----------+
|[21.0,6.0,160.0]  |110.0     |
|[21.0,6.0,160.0]  |110.0     |
|[228.0,4.0,108.0] |93.0      |
|[214.0,6.0,258.0] |110.0     |
|[187.0,8.0,360.0] |175.0     |
|[181.0,6.0,225.0] |105.0     |
|[143.0,8.0,360.0] |245.0     |
|[244.0,4.0,1467.0]|62.0      |
|[228.0,4.0,1408.0]|95.0      |
|[192.0,6.0,1676.0]|123.0     |
|[178.0,6.0,1676.0]|123.0     |
|[164.0,8.0,2758.0]|180.0     |
|[173.0,8.0,2758.0]|180.0     |
|[152.0,8.0,2758.0]|180.0     |
|[104.0,8.0,472.0] |205.0     |
|[104.0,8.0,460.0] |215.0     |
|[147.0,8.0,440.0] |230.0     |
|[324.0,4.0,787.0] |66.0      |
|[304.0,4.0,757.0] |52.0      |
|[339.0,4.0,711.0] |65.0      |
+------------------+----------+
only showing top 20 rows



In [35]:
carros_treino, carros_teste = carrosrf.randomSplit([0.75,0.25])

In [36]:
print(carros_treino.count(), carros_teste.count())

24 8


In [37]:
from pyspark.ml.regression import LinearRegression

In [38]:
regressao = LinearRegression(featuresCol='independente', labelCol='dependente',maxIter=1000,loss='squaredError', standardization=True)
modelo = regressao.fit(carros_treino)

In [39]:
previsao = modelo.transform(carros_teste)
previsao.select('dependente','prediction').show() # dependente = y_teste, prediction = previsao

+----------+------------------+
|dependente|        prediction|
+----------+------------------+
|     110.0|173.25798917600602|
|     215.0|217.93650864291666|
|     205.0| 217.8239091994631|
|     245.0|214.85582996243474|
|     180.0| 187.8643905757507|
|     180.0|186.58799187887973|
|     175.0|207.10360491166443|
|     175.0| 206.0191630463353|
+----------+------------------+



métricas

In [40]:
from pyspark.ml.evaluation import RegressionEvaluator

In [42]:
avaliacao = RegressionEvaluator(predictionCol='prediction',labelCol='dependente',metricName='rmse')
rmse = avaliacao.evaluate(previsao)
print(rmse) #quanto mais proxima de zero melhor

29.961225176373507
